In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
with open(r'.\Reviews_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

y_train = keras.utils.to_categorical(y_train-1, num_classes=5)
y_val = keras.utils.to_categorical(y_val-1, num_classes=5)
y_test = keras.utils.to_categorical(y_test-1, num_classes=5)

with open(r'.\embeddingMatrix_Reviews.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_REVIEW_LEN = 250 # maximum words in a review
num_classes = 5

In [3]:
def build_model(hp, min_layers, max_layers, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                test_activations, use_BatchNormalization, use_Dropout, filters_min_value, filters_max_value, filters_step, same_filters):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
    
    model = keras.Sequential()
    model.add(embedding_layer)

    num_layers = hp.Int('num_layers', min_layers, max_layers)

    if test_activations:
            activation = hp.Choice(f'activation', ['softplus', 'softsign', 'relu', 'tanh'])
    else:
        activation = 'relu'  # Default activation

    if test_initializers:
        kernel_initializer = hp.Choice(f'kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
    else:
        kernel_initializer = 'he_uniform'

    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    # Hyperparameters for the number of layers
    for i in range(num_layers):
        if same_filters:
            filters = filters_min_value
        else:
            filters = hp.Int(f'filters_{i}', min_value=filters_min_value, max_value=filters_max_value, step=filters_step)

        model.add(layers.Conv1D(
            filters=filters,
            kernel_size=5,
            activation=activation,
            kernel_initializer=kernel_initializer,
            kernel_regularizer=kernel_regularizer,
            padding='same'
        ))

        if use_BatchNormalization and hp.Boolean(f'batch_norm_{i}'):
            model.add(layers.BatchNormalization())

        model.add(layers.Dropout(rate=0.2))

        if use_Dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_{i}', [0.0, 0.2, 0.4])))

        model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
NUM_EPOCHS = 12
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/CNN/CNN_V2_3Layer_Optimizer_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/CNN/CNN_V2_3Layer_Optimizer_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model(hp, min_layers=3, max_layers=3, test_optimizers=True, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l1', test_learning_rate=False, test_activations=False, use_BatchNormalization=False, 
                           use_Dropout=False, filters_min_value=128, filters_max_value=128, filters_step=128, same_filters=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [3]:
def build_model_pd(hp, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                test_activations):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
    
    model = keras.Sequential()
    model.add(embedding_layer)

    if test_activations:
        activation_choice = hp.Choice('activation', ['softplus', 'softsign', 'relu', 'tanh'])
        activation = tf.keras.activations.get(activation_choice)
    else:
        activation = 'relu'  # Default activation

    if test_initializers:
        initializer_choice = hp.Choice('kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
        kernel_initializer = tf.keras.initializers.get(initializer_choice)
    else:
        kernel_initializer = 'he_uniform'

    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    model.add(layers.Conv1D(filters=96, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Conv1D(filters=96, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(rate=0.2))
    
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(rate=0.2))

    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
NUM_EPOCHS = 25
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/CNN/CNN_V6_3Layer_LearningRate_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/CNN/CNN_V6_3Layer_LearningRate_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_pd(hp, test_optimizers=False, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l2', test_learning_rate=True, test_activations=False),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [7]:
def build_model_pd2(hp, test_optimizers, test_initializers, test_regularizer, regularizer_choice, test_learning_rate,
                   test_activations, test_batch_norm):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)
    
    if test_activations:
        activation_choice = hp.Choice('activation', ['softplus', 'softsign', 'relu', 'tanh'])
        activation = tf.keras.activations.get(activation_choice)
    else:
        activation = 'relu'
    
    if test_initializers:
        initializer_choice = hp.Choice('kernel_initializer', ['glorot_uniform', 'he_uniform', 'random_uniform'])
        kernel_initializer = tf.keras.initializers.get(initializer_choice)
    else:
        kernel_initializer = 'he_uniform'
    
    if test_regularizer:
        if regularizer_choice == 'l1':
            kernel_regularizer = keras.regularizers.L1(l1=hp.Choice('l1_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l2':
            kernel_regularizer = keras.regularizers.L2(l2=hp.Choice('l2_factor', [1e-4, 1e-2]))
        elif regularizer_choice == 'l1_l2':
            kernel_regularizer = keras.regularizers.L1L2(l1=hp.Choice('l1_l2_l1_factor', [1e-4, 1e-2]),
                                    l2=hp.Choice('l1_l2_l2_factor', [1e-4, 1e-2]))
    else:
        kernel_regularizer = None

    # Function to add a convolutional block
    def add_conv_block(filters, dropout_rate=0.2):
        model.add(layers.Conv1D(filters=filters, kernel_size=5, activation=activation,
                kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
        if test_batch_norm:
            use_bn = hp.Boolean(f'batch_norm_after_conv_{filters}')
            if use_bn:
                model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling1D(pool_size=2))
        if dropout_rate > 0:
            model.add(layers.Dropout(rate=dropout_rate))

    # Add convolutional blocks
    add_conv_block(96)
    add_conv_block(96)
    add_conv_block(64, dropout_rate=0)

    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        if test_learning_rate:
            learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])
        else:
            learning_rate = 0.001
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
NUM_EPOCHS = 25
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/CNN/CNN_V7_3Layer_BatchNormalization_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/CNN/CNN_V7_3Layer_BatchNormalization_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_pd2(hp, test_optimizers=False, test_initializers=False, test_regularizer=False, 
                           regularizer_choice='l2', test_learning_rate=False, test_activations=False, test_batch_norm=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [14]:
def build_model_do(hp, use_dropout):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True)
    
    model = keras.Sequential()
    model.add(embedding_layer)

    activation = 'relu'  # Default activation
    kernel_initializer = 'he_uniform'
    kernel_regularizer = None

    model.add(layers.Conv1D(filters=96, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_1', [0.0, 0.2, 0.4])))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Conv1D(filters=96, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_2', [0.0, 0.2, 0.4])))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation=activation,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, padding='same'))
    model.add(layers.BatchNormalization())
    if use_dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_3', [0.0, 0.2, 0.4])))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
NUM_EPOCHS = 15
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/CNN/CNN_V8_3Layer_DropOut_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/CNN/CNN_V8_3Layer_DropOut_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model_do(hp, use_dropout=True),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [16]:
tuner.search(x=X_train,
             y=y_train,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, y_val))

Trial 27 Complete [00h 01m 17s]
val_loss: 1.0673409700393677

Best val_loss So Far: 1.0247337818145752
Total elapsed time: 00h 30m 39s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'dropout_rate_1': 0.0, 'dropout_rate_2': 0.4, 'dropout_rate_3': 0.2}

In [18]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 300)          16058400  
                                                                 
 conv1d (Conv1D)             (None, 250, 96)           144096    
                                                                 
 dropout (Dropout)           (None, 250, 96)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 125, 96)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 125, 96)           46176     
                                                                 
 dropout_1 (Dropout)         (None, 125, 96)           0         
                                                        

In [21]:
tuner.results_summary()

Results summary
Results in KerasTuner_Logs/CNN/CNN_V5_3Layer_RegularizerL2_2024-07-13_13-28-40\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0000 summary
Hyperparameters:
l2_factor: 0.0001
Score: 1.1286063194274902

Trial 0001 summary
Hyperparameters:
l2_factor: 0.01
Score: 1.6093945503234863
